#Import Libraries


Add in more as needed

In [ ]:
#installing libraries
print("Importing libraries...")

import tensorflow as tf
import random, os
import shutil
from matplotlib.image import imread
import sklearn
from sklearn.model_selection import train_test_split
import os
import random
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from keras.applications.mobilenet import MobileNet
from keras.metrics import AUC

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score



#installing datasets
print("Downloading data...")

!wget https://www.dropbox.com/s/fedqcdt4o0m2oxp/X.npy &> /dev/null
!wget https://www.dropbox.com/s/h7xh92w1w7px30a/X_g.npy &> /dev/null
!wget https://www.dropbox.com/s/grn9brfvzx74c8a/y.npy &> /dev/null

#installing the diabetic retinopathy data set
print("\ninstalling the diabetic retinopathy dataset...\n\n")
!wget https://www.dropbox.com/s/8qigh2prdf116a3/dr224.zip?dl=0


#installing commands for python with data science (commands are found in the dropbox file "simplepythonfunctions_trishay"))
print("\ninstalling commands for python with data science...\n\n")
!wget https://www.dropbox.com/s/qm4lww85w1vrtve/retrain.py?dl=0

# Prepare data
os.makedirs("methods/js", exist_ok=True)
!wget -O static/js/skin_cancer_diagnosis_script.js 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20(Healthcare%20B)%20Skin%20Cancer%20Diagnosis/skin_cancer_diagnosis_script.js' &> /dev/null
output = 'static/js/skin_cancer_diagnosis_script.js'

print("Done!")


In [ ]:
#Importing more libraries
print('Importing image.io...')
import imageio
print('Done!')

#Unzip Dataset

In [ ]:
#Unzip the dataset

!unzip dr224.zip?dl=0

#Assign the train.csv file binary/identification label


In [ ]:
# Add an additional column, mapping to the type

df = pd.read_csv(r'train.csv')

diagnosis_dict_binary = {
    0: 'No_DR',
    1: 'DR',
    2: 'DR',
    3: 'DR',
    4: 'DR'
}

diagnosis_dict = {
    0: 'No_DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferate_DR',
}


df['binary_type'] =  df['diagnosis'].map(diagnosis_dict_binary.get)
df['type'] = df['diagnosis'].map(diagnosis_dict.get)
df.head()
display(df)

#Data Wrangling

Get it into a useful format

- A list of images (np.array or pd)
- Corresonding list of diagnosis - binarizing for 0(healthy) or 1(DR)

Options:
- **Import in images from each folder, append them to the array, and add that many instances of 0 or 1 to the list of diagnoses**
- Import in all the images randomly, use the train.csv or etc. to correlate by image name


The specific images you need:
- All the healthy
- All of the Severe, Proliferative, random half of Moderate

OR
- Half of healthy
- All of severe + proliferative

Aim for balanced dataset!

In [ ]:
# Create an empty list to store the images
healthy_list = []
moderate_list = []
severe_list = []
proliferate_list = []

# Get the list of files in the directory
healthy_files = os.listdir('/content/gaussian_filtered_images/gaussian_filtered_images/No_DR')
moderate_files = os.listdir('/content/gaussian_filtered_images/gaussian_filtered_images/Moderate')
severe_files = os.listdir('/content/gaussian_filtered_images/gaussian_filtered_images/Severe')
proliferate_files = os.listdir('/content/gaussian_filtered_images/gaussian_filtered_images/Proliferate_DR')

# Iterate over the files and append them to the healthy_list
for file in healthy_files:
    file_path = os.path.join('/content/gaussian_filtered_images/gaussian_filtered_images/No_DR', file)
    healthy_list.append(imageio.imread(file_path))

# Iterate over the files and append them to the moderate_list
for file in moderate_files:
    file_path = os.path.join('/content/gaussian_filtered_images/gaussian_filtered_images/Moderate', file)
    moderate_list.append(imageio.imread(file_path))

# Iterate over the files and append them to the severe_list
for file in severe_files:
    file_path = os.path.join('/content/gaussian_filtered_images/gaussian_filtered_images/Severe', file)
    severe_list.append(imageio.imread(file_path))

# Iterate over the files and append them to the proliferate_list
for file in proliferate_files:
    file_path = os.path.join('/content/gaussian_filtered_images/gaussian_filtered_images/Proliferate_DR', file)
    proliferate_list.append(imageio.imread(file_path))

# Convert the list of images to a NumPy array
healthy_array = np.array(healthy_list)
moderate_array = np.array(moderate_list)
severe_array = np.array(severe_list)
proliferate_array = np.array(proliferate_list)

In [ ]:
#binary code for the list 0 = no_dr, 1 = dr

moderate_copy = moderate_array
random.shuffle(moderate_copy)

moderate_copy_half = moderate_copy[:len(moderate_copy) // 2]

yes_dr1 = np.concatenate((moderate_copy_half, severe_array, proliferate_array))


#toning down the size of the healthy array from 1804 to 987 to equal the combined size of the yes_dr1 array
healthy_copy = healthy_array
random.shuffle(healthy_copy)

healthy_copy_half = healthy_copy[0:987]

no_dr0 = healthy_copy_half

In [ ]:
#creating a giant list with all the instances of 0s and 1s

# Combine the arrays
combined_array = np.concatenate([no_dr0, yes_dr1])

# Create a label array with the binary labels
zeroes = np.zeros(987)
ones = np.ones(987)
label_array = np.concatenate([zeroes, ones])

In [ ]:
#Stack arrays to make a 2D array ----> save for later

# combined_array_2D = np.column_stack([combined_array, label_array])

# Data Exploration

- size: # of images, # of pixels in each, is it the same for all
- imshow to print out a few examples
- bar graph for showing balanced dataset
- relevant statistics: avg, std, etc.

In [ ]:
# Verifying that all images are the same size -- 224 x 224

same_size = True
reference_shape = combined_array[0].shape
for i in range(1, len(combined_array)):
    if combined_array[i].shape != reference_shape:
        same_size = False
        break

if same_size:
    print("All images are the same size")

    height, width = combined_array.shape[:2]
    print("Image size: {} x {}".format(height, width))
else:
    print("Not all images are the same size")

In [ ]:
#Verifying that there are the same number of images in both sets -- balanced dataset

num_images1 = no_dr0.shape[0]
num_images2 = yes_dr1.shape[0]

print(num_images1)
print(num_images2)

In [ ]:
#Printing out a few image examples

plt.imshow(combined_array[1])
plt.show()

plt.imshow(combined_array[467])
plt.show()

In [ ]:
#Matplotlib Bar Graph Construction -- Verify Balanced Dataset

# Count the number of no_dr0 and yes_dr1 images in the combined_array
num_no_dr0 = no_dr0.shape[0]
num_yes_dr1 = yes_dr1.shape[0]

# Create a figure and axis
fig, ax = plt.subplots()

# Create a bar plot with two bars, one for no_dr0 and one for yes_dr1
ax.bar(0, num_no_dr0, label="no_dr0")
ax.bar(1, num_yes_dr1, label="yes_dr1")

# Set the x-axis tick labels
ax.set_xticks([0, 1])
ax.set_xticklabels(["no_dr0", "yes_dr1"])

# Set the y-axis tick labels
ax.set_ylabel("Number of images")

# Add labels to the bar graphs
ax.text(0, num_no_dr0 + 50, str(num_no_dr0), ha="center", va="bottom", fontsize=14, color="red", weight="bold")
ax.text(1, num_yes_dr1 + 50, str(num_yes_dr1), ha="center", va="bottom", fontsize=14, color="red", weight="bold")

# Add a legend
ax.legend()

# Show the plot
plt.show()


#Machine Learning Algorithm

Note: Experimental Only


In [ ]:
# Baseline Models To Implement:

# KNN Algorithm: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

# Logistic Regression: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

# Decision Tree Classifier: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

# ----------------

# If time: for loop and graph the effect of different n_neighbors or dt depth for example - on the accuracy


In [ ]:
# Necessary imports
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
print("Importing additional libraries...")

# Extract the features and labels from the combined_array_2D array:
print("Extracting features and labels from the combined 2D array...")
X = combined_array # features (equalized images)
y = label_array # labels (diagnosis of DR)

# Split the data into training and test sets
print("Splitting the data into training and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Flatten the images
print("Flattening Images...")
X = X.flatten()
X_train = X_train.reshape(-1, 50176)
X_test = X_test.reshape(-1, 50176)

# create for loop to change the number of neighbors from 3 - 35 (odd numbers only), graph at the end the number of neighbors vs the accuracy for each
for i in range(3, 36, 2):
    # Define the model
    print("Defining the model...")
    model = KNeighborsClassifier(n_neighbors=i)
    # Train the model
    print("Training the model...")
    model.fit(X_train, y_train)
    # Make predictions
    print("Creating Predictions...")
    y_pred = model.predict(X_test)
    # Evaluate the model
    print("Evaluating the model...")
    accuracy = model.score(X_test, y_test)
    print("Printing accuracy for KNN algorithm...")
    print("for a k value of " + str(i) +" the accuracy was:" )
    print("\nAccuracy: {:.2f}".format(accuracy))
